In [ ]:
from PIL import Image 
import PIL 
import math
import numpy as np
import math
          

In [ ]:

#this function calculate gaussian filter based on filter size and sigma value    
def calcualteGaussianFilter(filterSize,sigma):
    filterMatrix = []
    k = int((filterSize-1)/2)
    for i in range(-k,k+1):
        for j in range(-k,k+1):
            a = 1/(2*math.pi*sigma**2)
            b = math.e**-((i**2+j**2)/(2*sigma**2))
            filterMatrix.append(round(a*b,3))
    filterMatrix = np.array(filterMatrix).reshape(filterSize,filterSize)                 
    return filterMatrix
    
       

      
# this function will be used for both of mean and gaussing because onşy difference between them is kernel. 
# here we extract the R,G and B matrices of images  and apply filter (gaussian or mean kernal) all of channels and merge them,create output images
def blurring(image,filterSize,blurringType,sigma=0):
    width, height = image.size
    image = image.convert('RGB')
    filteredImage = image
    pixels = filteredImage.load()
    image = np.array(image)
    image = image.transpose((2,0,1)).reshape(3,-1)
    rChannel,gChannel,bChannel = image[0].reshape(height,width),image[1].reshape(height,width),image[2].reshape(height,width)
    if(blurringType=="mean"):
        filterMatrix = np.full((filterSize,filterSize),1/filterSize**2)
    elif(blurringType=="gaussian"):
        filterMatrix = calcualteGaussianFilter(filterSize,sigma)      
    
    start = int(filterSize/2)
    heightRange = height - start
    widthRange = width - start
        
    for i in range(start,heightRange):
      for j in range(start,widthRange):
         index1 = int(filterSize/2)
         index2 = int((filterSize/2)+1.0)
         newR = np.sum(np.multiply(filterMatrix,rChannel[i-index1:i+index2,j-index1:j+index2]))
         newG = np.sum(np.multiply(filterMatrix,gChannel[i-index1:i+index2,j-index1:j+index2]))
         newB = np.sum(np.multiply(filterMatrix,bChannel[i-index1:i+index2,j-index1:j+index2]))
         coordinates = x,y = (j,i) 
         pixels[coordinates] = (int(newR),int(newG),int(newB)) 
         
       
    filteredImage = filteredImage.crop((start,start,width-start,height-start))# because of no padding our images will be a bit smaller
    filteredImage.show()     


# this function for calculating mean and standard deviation of quadrants for kuwahaa filter
# here we use value V (HSV of images ) for calculate standard deviation of quadrants and 
# R,G or B channel for mean calculating
def calculateMeanAndStandartDeviation(value,filterSize,colorChannel):
    firstIndex = int(filterSize/2)
    secondIndex = int((filterSize/2)+1.0)
    
    Q1 = value[0:secondIndex,firstIndex:]
    Q2 = value[0:secondIndex,0:secondIndex]
    Q3 = value[firstIndex:,0:secondIndex]
    Q4 = value[firstIndex:,firstIndex:]
    QArrays = [Q1,Q2,Q3,Q4]

    standartDeviations=[]
    standartDeviations.extend([np.std(Q1),np.std(Q2),np.std(Q3),np.std(Q4)])
    minStd = np.array(standartDeviations).min()
    summary = 0

    for i in QArrays:
        if(np.std(i)==minStd):
            if(np.array_equal(i, Q1)):
                summary += np.mean(colorChannel[0:secondIndex,firstIndex:])
            elif (np.array_equal(i, Q2)):
                summary += np.mean(colorChannel[0:secondIndex,0:secondIndex])
            elif (np.array_equal(i, Q3)):
                summary += np.mean(colorChannel[firstIndex:,0:secondIndex])
            elif (np.array_equal(i, Q4)):
                summary += np.mean(colorChannel[firstIndex:,firstIndex:])
    return summary
    
    
    
# we use V which is come HSV for calculating standard deviation and use R,G and B for calcualte quadrant mean
# we extract R,G, and B matrices of images and  do same process (std and mean calcualtion) all of channel (R,G,B)  
def kuwaharaFilter(image,filterSize):
    width, height = image.size
    copyImage = image
    copyImage = copyImage.convert('HSV')
    copyImage = np.array(copyImage)
    copyImage = copyImage.transpose((2,0,1)).reshape(3,-1)
    value = copyImage[2].reshape(height,width)
    image = image.convert('RGB')
    filteredImage = image
    pixels = filteredImage.load()
    image = np.array(image)
    image = image.transpose((2,0,1)).reshape(3,-1)
    rChannel,gChannel,bChannel = image[0].reshape(height,width),image[1].reshape(height,width),image[2].reshape(height,width)
    
    start = int(filterSize/2)
    heightRange = height - start
    widthRange = width - start
    
        
    for i in range(start,heightRange):
      for j in range(start,widthRange):
          
          index1 = int(filterSize/2)
          index2 = int((filterSize/2)+1.0)
          newR = calculateMeanAndStandartDeviation(value[i-index1:i+index2,j-index1:j+index2],filterSize,rChannel[i-index1:i+index2,j-index1:j+index2])
          newG = calculateMeanAndStandartDeviation(value[i-index1:i+index2,j-index1:j+index2],filterSize,gChannel[i-index1:i+index2,j-index1:j+index2])
          newB = calculateMeanAndStandartDeviation(value[i-index1:i+index2,j-index1:j+index2],filterSize,bChannel[i-index1:i+index2,j-index1:j+index2])
          coordinates = x,y = (j,i) 
          pixels[coordinates] = (int(newR),int(newG),int(newB))
          

    filteredImage = filteredImage.convert('RGB')    
    filteredImage = filteredImage.crop((start,start,width-start,height-start)) # because of no padding our images will be a bit smaller
    filteredImage.show()      



In [ ]:
image = Image.open("image.png")
#blurring(image, 3,"gaussian",0.5)
#blurring(image, 3,"mean")
kuwaharaFilter(image, 7)